In [2]:
!pwd

/content


In [3]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/image_classification/

/content/drive/MyDrive/kaggle/image_classification


In [4]:
%cd 10_monkey_species/

/content/drive/MyDrive/kaggle/image_classification/10_monkey_species


In [5]:
!ls

monkey_labels.txt  test  train


In [22]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [23]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [24]:
train_path = 'train'
valid_path = 'test'

In [25]:
# add preprocessing layer to the front of Inception_v3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [26]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [27]:
# useful for getting number of classes
folders = glob('train/*')

In [28]:
# our layers - you can add more if you want
x = Flatten()(inception.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [29]:
# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [30]:
# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [31]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [32]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1098 images belonging to 10 classes.


In [34]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 272 images belonging to 10 classes.


In [35]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=9,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/9
35/35 [==============================] - 55s 1s/step - loss: 2.3540 - accuracy: 0.8051 - val_loss: 1.0557 - val_accuracy: 0.9228
Epoch 2/9
35/35 [==============================] - 48s 1s/step - loss: 0.4718 - accuracy: 0.9554 - val_loss: 0.4309 - val_accuracy: 0.9559
Epoch 3/9
35/35 [==============================] - 47s 1s/step - loss: 0.2934 - accuracy: 0.9627 - val_loss: 0.3292 - val_accuracy: 0.9522
Epoch 4/9
35/35 [==============================] - 48s 1s/step - loss: 0.3383 - accuracy: 0.9681 - val_loss: 0.4360 - val_accuracy: 0.9559
Epoch 5/9
35/35 [==============================] - 48s 1s/step - loss: 0.2872 - accuracy: 0.9736 - val_loss: 0.8603 - val_accuracy: 0.9522
Epoch 6/9
35/35 [==============================] - 48s 1s/step - loss: 0.2365 - accuracy: 0.9827 - val_loss: 0.7323 - val_accuracy: 0.9485
Epoch 7/9
35/35 [==============================] - 47s 1s/step - loss: 0.1242 - accuracy: 0.9845 - val_loss: 0.4829 - val_accuracy: 0.9669
Epoch 8/9
35/35 [==========